In [ ]:
# ==========================================
# 1. SETUP ENVIRONMENT & DATA
# ==========================================
from google.colab import drive
import os
import gdown

# 1. Mount Google Drive (To save your models safely)
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Create a folder in Drive for checkpoints
# Change "Duality_Project" to whatever folder name you prefer
DRIVE_ROOT = "/content/drive/MyDrive/Duality_Project"
os.makedirs(f"{DRIVE_ROOT}/checkpoints", exist_ok=True)
print(f"✓ Checkpoints will be saved to: {DRIVE_ROOT}/checkpoints")

# 2. Install Libraries
print("\nInstalling libraries... (This may take a minute)")
!pip install -q segmentation-models-pytorch albumentations gdown wandb

# 3. Download Data (Using your Direct Link)
print("\nDownloading Dataset...")
url = "https://storage.googleapis.com/duality-public-share/Hackathons/Duality%20Hackathon/Offroad_Segmentation_Training_Dataset.zip"
output_zip = "/content/dataset.zip"

if not os.path.exists(output_zip):
    gdown.download(url, output_zip, quiet=False)
    print("Unzipping dataset...")
    !unzip -q {output_zip} -d /content/data
    print("✓ Data extracted to /content/data")
else:
    print("✓ Dataset already exists, skipping download.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Checkpoints will be saved to: /content/drive/MyDrive/Duality_Project/checkpoints

Installing libraries... (This may take a minute)



Downloading...
From: https://storage.googleapis.com/duality-public-share/Hackathons/Duality%20Hackathon/Offroad_Segmentation_Training_Dataset.zip
To: /content/dataset.zip
100%|██████████| 2.79G/2.79G [01:58<00:00, 23.7MB/s]


Unzipping dataset...
✓ Data extracted to /content/data


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import Counter
import cv2
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import wandb
import json
from datetime import datetime
import shutil
import time
import zipfile

# ==========================================
# CONFIGURATION
# ==========================================
class Config:
    PROJECT_NAME = "duality-offroad-segmentation"
    RUN_NAME = f"unet-resnet34-colab-{datetime.now().strftime('%Y%m%d-%H%M')}"

    # --- PATHS (ADJUSTED FOR COLAB) ---
    DATASET_ROOT = "/content/data/Offroad_Segmentation_Training_Dataset"

    # Save directly to Drive so we don't lose data if Colab crashes
    CHECKPOINT_DIR = f"/content/drive/MyDrive/Duality_Project/checkpoints/{RUN_NAME}"
    OUTPUT_DIR = f"/content/outputs/{RUN_NAME}"

    TRAIN_IMG_DIR = f"{DATASET_ROOT}/train/Color_Images"
    TRAIN_MASK_DIR = f"{DATASET_ROOT}/train/Segmentation"
    VAL_IMG_DIR = f"{DATASET_ROOT}/val/Color_Images"
    VAL_MASK_DIR = f"{DATASET_ROOT}/val/Segmentation"

    # --- MODEL & TRAINING ---
    ENCODER = "resnet34"
    ENCODER_WEIGHTS = "imagenet"
    NUM_CLASSES = 10

    # Reduced Batch Size for Colab T4 GPU (Prevents Out-Of-Memory)
    BATCH_SIZE = 8
    NUM_EPOCHS = 50
    LEARNING_RATE = 5e-4
    WEIGHT_DECAY = 1e-4
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    NUM_WORKERS = 2
    USE_AMP = True  # Mixed Precision (Faster)

    # Class Definitions
    CLASS_MAPPING = {
        100: 0, 200: 1, 300: 2, 500: 3, 550: 4,
        600: 5, 700: 6, 800: 7, 7100: 8, 10000: 9
    }
    CLASS_NAMES = ["Trees", "Lush Bushes", "Dry Grass", "Dry Bushes",
                   "Ground Clutter", "Flowers", "Logs", "Rocks",
                   "Landscape", "Sky"]

    USE_WANDB = False
    SAVE_FREQUENCY = 5

# Create directories
Path(Config.CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)
Path(Config.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print(f"✓ Config Loaded. Saving to: {Config.CHECKPOINT_DIR}")

# ==========================================
# WANDB LOGIN (Interactive)
# ==========================================
if Config.USE_WANDB:
    print("\n--- WandB Login ---")
    try:
        from google.colab import userdata
        key = userdata.get('WANDB_API_KEY')
        wandb.login(key=key)
    except:
        print("Please paste your WandB API Key below (or press Enter to skip):")
        wandb.login()

    try:
        wandb.init(project=Config.PROJECT_NAME, name=Config.RUN_NAME, config={
            "encoder": Config.ENCODER, "batch_size": Config.BATCH_SIZE, "lr": Config.LEARNING_RATE
        })
        print("✓ WandB Initialized")
    except:
        print("⚠ WandB skipped (No key provided)")
        Config.USE_WANDB = False

✓ Config Loaded. Saving to: /content/drive/MyDrive/Duality_Project/checkpoints/unet-resnet34-colab-20260204-0618


In [ ]:
# ==========================================
# AUGMENTATIONS (The "Heavy" Stuff)
# ==========================================
def get_training_augmentation():
    return A.Compose([
        # --- GEOMETRIC ---
        A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2), # Good for top-down terrain maps

        # --- LIGHTING (They handle day/night diffs) ---
        A.RandomBrightnessContrast(p=0.5),
        A.RandomGamma(p=0.5),

        # --- DESTRUCTIVE (OPTIONAL) ---

        # A.OneOf([
        #     A.MotionBlur(p=1),   <-- COMMENT OUT
        #     A.MedianBlur(p=1),   <-- COMMENT OUT
        # ], p=0.3),

        # A.CoarseDropout(..., p=0.3), <-- COMMENT OUT

        # --- ZOOM/CROP (Keep but make milder) ---
        # scale=(0.7, 1.0) is aggressive. Change to 0.85
        A.RandomResizedCrop(size=(544, 960), scale=(0.85, 1.0), p=0.5),

        A.Resize(544, 960),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

def get_validation_augmentation():
    return A.Compose([
        A.Resize(544, 960),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

# ==========================================
# DATASET CLASS
# ==========================================
class DesertSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        self.image_paths = sorted(list(self.image_dir.glob("*.png")) + list(self.image_dir.glob("*.jpg")))

    def __len__(self): return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_dir / img_path.name

        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(str(mask_path), cv2.IMREAD_UNCHANGED)

        # Remap IDs
        new_mask = np.zeros_like(mask, dtype=np.int64)
        for old_id, new_id in Config.CLASS_MAPPING.items():
            new_mask[mask == old_id] = new_id

        if self.transform:
            aug = self.transform(image=image, mask=new_mask)
            image, new_mask = aug['image'], aug['mask']

        return image, new_mask.long()

# ==========================================
# DATA ANALYSIS & SAMPLING
# ==========================================
def check_class_distribution(mask_dir, dataset_name="Dataset"):
    print(f"\nAnalyzing {dataset_name} distribution...")
    class_counts = Counter()
    mask_paths = list(Path(mask_dir).glob("*.png")) + list(Path(mask_dir).glob("*.jpg"))

    for mask_path in tqdm(mask_paths, leave=False):
        mask = cv2.imread(str(mask_path), cv2.IMREAD_UNCHANGED)
        remapped = np.zeros_like(mask, dtype=np.int64)
        for old_id, new_id in Config.CLASS_MAPPING.items():
            remapped[mask == old_id] = new_id

        unique, counts = np.unique(remapped, return_counts=True)
        for cls, count in zip(unique, counts):
            class_counts[cls] += count

    total_pixels = sum(class_counts.values())
    print("-" * 70)
    for i, name in enumerate(Config.CLASS_NAMES):
        pct = (class_counts.get(i, 0) / total_pixels) * 100
        print(f"{name:15s}: {class_counts.get(i,0):12,d} pixels ({pct:5.2f}%)")
    return class_counts

# Run Analysis
train_dist = check_class_distribution(Config.TRAIN_MASK_DIR, "TRAIN")


Analyzing TRAIN distribution...


  0%|          | 0/2857 [00:00<?, ?it/s]

----------------------------------------------------------------------
Trees          :   52,331,525 pixels ( 3.53%)
Lush Bushes    :   87,892,776 pixels ( 5.93%)
Dry Grass      :  279,430,843 pixels (18.87%)
Dry Bushes     :   16,268,713 pixels ( 1.10%)
Ground Clutter :   65,082,995 pixels ( 4.39%)
Flowers        :   41,585,811 pixels ( 2.81%)
Logs           :    1,153,995 pixels ( 0.08%)
Rocks          :   17,743,187 pixels ( 1.20%)
Landscape      :  362,120,221 pixels (24.45%)
Sky            :  557,458,734 pixels (37.64%)


In [ ]:
# --- WEIGHTED SAMPLER ---
# This ensures we see "Flowers", "Dry Bushes", and "Logs" more often
def create_weighted_sampler(mask_dir):
    print("\nCreating Weighted Sampler (Boosting Rare Classes)...")
    mask_paths = sorted(list(Path(mask_dir).glob("*.png")) + list(Path(mask_dir).glob("*.jpg")))
    sample_weights = []

    for mask_path in tqdm(mask_paths):
        mask = cv2.imread(str(mask_path), cv2.IMREAD_UNCHANGED)
        # Boost Dry Bushes (500), Flowers (600), Logs (700), and Rocks (800)
        if (mask == 500).any() or (mask == 600).any() or (mask == 700).any() or (mask == 800).any():
            sample_weights.append(3.0) # 3x Boost
        else:
            sample_weights.append(1.0)

    return WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_sampler = create_weighted_sampler(Config.TRAIN_MASK_DIR)

# Setup Loaders
train_ds = DesertSegmentationDataset(Config.TRAIN_IMG_DIR, Config.TRAIN_MASK_DIR, transform=get_training_augmentation())
val_ds = DesertSegmentationDataset(Config.VAL_IMG_DIR, Config.VAL_MASK_DIR, transform=get_validation_augmentation())

train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, sampler=train_sampler, num_workers=Config.NUM_WORKERS)
val_loader = DataLoader(val_ds, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=Config.NUM_WORKERS)

# ==========================================
# LOSS FUNCTION & OPTIMIZER
# ==========================================
# Manual class weights to punish missing rare items
class_weights = torch.tensor([
    1.1, 0.9, 0.6, 3.0, 1.2, 2.5, 1.8, 1.5, 0.5, 0.4
]).to(Config.DEVICE)

print("\nActive Class Weights:")
for name, w in zip(Config.CLASS_NAMES, class_weights):
    print(f"  {name:15s}: {w:.1f}")

class CombinedLoss(nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.dice_weight = 0.75
        self.ce_weight = 0.25
        self.ce = nn.CrossEntropyLoss(weight=weights)

    def dice_loss(self, pred, target, smooth=1e-6):
        pred = torch.softmax(pred, dim=1)
        target_one_hot = torch.zeros_like(pred)
        target_one_hot.scatter_(1, target.unsqueeze(1), 1)

        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        dice = (2. * intersection + smooth) / (union + smooth)
        return 1 - dice.mean()

    def forward(self, pred, target):
        return self.dice_weight * self.dice_loss(pred, target) + self.ce_weight * self.ce(pred, target)


Creating Weighted Sampler (Boosting Rare Classes)...


  0%|          | 0/2857 [00:00<?, ?it/s]


Active Class Weights:
  Trees          : 1.1
  Lush Bushes    : 0.9
  Dry Grass      : 0.6
  Dry Bushes     : 3.0
  Ground Clutter : 1.2
  Flowers        : 2.5
  Logs           : 1.8
  Rocks          : 1.5
  Landscape      : 0.5
  Sky            : 0.4


In [ ]:
# ==========================================
# TRAINING LOOPS
# ==========================================
model = smp.Unet(encoder_name=Config.ENCODER, encoder_weights=Config.ENCODER_WEIGHTS, classes=Config.NUM_CLASSES).to(Config.DEVICE)
criterion = CombinedLoss(class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=Config.USE_AMP)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=Config.LEARNING_RATE, epochs=Config.NUM_EPOCHS,
    steps_per_epoch=len(train_loader), pct_start=0.2, div_factor=10.0
)

def calculate_iou(pred, target):
    ious = []
    pred = pred.view(-1).cpu().numpy()
    target = target.view(-1).cpu().numpy()
    for cls in range(Config.NUM_CLASSES):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum()
        union = (pred_inds | target_inds).sum()
        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append(intersection / union)
    return np.array(ious)

print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70)

best_iou = 0.0

for epoch in range(Config.NUM_EPOCHS):
    model.train()
    epoch_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{Config.NUM_EPOCHS} [Train]")
    for images, masks in pbar:
        images, masks = images.to(Config.DEVICE), masks.to(Config.DEVICE)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=Config.USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, masks)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        epoch_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

        if Config.USE_WANDB: wandb.log({"batch_loss": loss.item()})

    # Validation
    model.eval()
    val_loss = 0
    class_iou_sum = np.zeros(Config.NUM_CLASSES)

    print(f"Validating Epoch {epoch+1}...")
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(Config.DEVICE), masks.to(Config.DEVICE)
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            batch_ious = calculate_iou(preds, masks)
            class_iou_sum += np.nan_to_num(batch_ious, nan=0.0)

    # Metrics
    avg_train_loss = epoch_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    avg_class_ious = class_iou_sum / len(val_loader)
    mean_val_iou = np.mean(avg_class_ious)

    print("-" * 70)
    print(f"Epoch {epoch+1} Summary:")
    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Mean IoU: {mean_val_iou:.4f}")
    print("\nPer-Class IoU:")
    for name, iou in zip(Config.CLASS_NAMES, avg_class_ious):
        print(f"  {name:15s}: {iou:.4f}")
    print("-" * 70)

    if Config.USE_WANDB:
        log_dict = {"epoch": epoch+1, "val_loss": avg_val_loss, "mean_iou": mean_val_iou}
        for name, iou in zip(Config.CLASS_NAMES, avg_class_ious):
            log_dict[f"class_iou/{name}"] = iou
        wandb.log(log_dict)

    # Save Checkpoints
    if mean_val_iou > best_iou:
        best_iou = mean_val_iou
        torch.save(model.state_dict(), f"{Config.CHECKPOINT_DIR}/best_model.pth")
        print(f"🏆 New Best Model Saved! (IoU: {best_iou:.4f})")

    if (epoch+1) % Config.SAVE_FREQUENCY == 0:
         torch.save(model.state_dict(), f"{Config.CHECKPOINT_DIR}/epoch_{epoch+1}.pth")
         print(f"✓ Routine checkpoint saved.")

print("\nTraining Complete!")
if Config.USE_WANDB: wandb.finish()


STARTING TRAINING


/tmp/ipython-input-1494039547.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=Config.USE_AMP)


Epoch 1/50 [Train]:   0%|          | 0/358 [00:00<?, ?it/s]

/tmp/ipython-input-1494039547.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=Config.USE_AMP):


Validating Epoch 1...
----------------------------------------------------------------------
Epoch 1 Summary:
Train Loss: 0.9428 | Val Loss: 0.7424 | Mean IoU: 0.3268

Per-Class IoU:
  Trees          : 0.4541
  Lush Bushes    : 0.3559
  Dry Grass      : 0.5651
  Dry Bushes     : 0.0022
  Ground Clutter : 0.2547
  Flowers        : 0.1516
  Logs           : 0.0000
  Rocks          : 0.0029
  Landscape      : 0.5512
  Sky            : 0.9300
----------------------------------------------------------------------
🏆 New Best Model Saved! (IoU: 0.3268)


...

Validating Epoch 36...
----------------------------------------------------------------------
Epoch 36 Summary:
Train Loss: 0.4404 | Val Loss: 0.4198 | Mean IoU: 0.5335

Per-Class IoU:
  Trees          : 0.6687
  Lush Bushes    : 0.6720
  Dry Grass      : 0.6834
  Dry Bushes     : 0.1633
  Ground Clutter : 0.4546
  Flowers        : 0.2312
  Logs           : 0.3200
  Rocks          : 0.5236
  Landscape      : 0.6784
  Sky            : 0.9393
----------------------------------------------------------------------
🏆 New Best Model Saved! (IoU: 0.5335)


Epoch 37/50 [Train]:   0%|          | 0/358 [00:00<?, ?it/s]

Validating Epoch 37...
----------------------------------------------------------------------
Epoch 37 Summary:
Train Loss: 0.4395 | Val Loss: 0.4198 | Mean IoU: 0.5321

Per-Class IoU:
  Trees          : 0.6667
  Lush Bushes    : 0.6712
  Dry Grass      : 0.6835
  Dry Bushes     : 0.1619
  Ground Clutter : 0.4549
  Flowers        : 0.2306
  Logs           : 0.3165
  Rocks          : 0.5264
  Landscape      : 0.6699
  Sky            : 0.9392
----------------------------------------------------------------------


Epoch 38/50 [Train]:   0%|          | 0/358 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# ==========================================
# FINAL REPORTING & BACKUP
# ==========================================
print("\n" + "="*70)
print("GENERATING FINAL REPORT")
print("="*70)

# 1. Plot Training Curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history['train_loss'], label='Train Loss')
axes[0, 0].plot(history['val_loss'], label='Val Loss')
axes[0, 0].set_xlabel('Epoch'); axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend(); axes[0, 0].grid(True)

# IoU (Jaccard)
# Note: In the new loss function, we tracked 'mean_iou' via print/wandb
# We reconstruct the list from the logs or use val_iou if available
if 'val_iou' in history and len(history['val_iou']) > 0:
    axes[0, 1].plot(history['val_iou'], label='Val IoU')
    axes[0, 1].set_xlabel('Epoch'); axes[0, 1].set_ylabel('IoU')
    axes[0, 1].set_title('Validation IoU')
    axes[0, 1].legend(); axes[0, 1].grid(True)

# Accuracy (If tracked)
if 'val_acc' in history and len(history['val_acc']) > 0:
    axes[1, 0].plot(history['val_acc'], label='Val Acc')
    axes[1, 0].set_title('Validation Accuracy')
    axes[1, 0].grid(True)
else:
    axes[1, 0].text(0.5, 0.5, 'Accuracy not tracked separately', ha='center')

# Final Per-Class IoU
if 'class_ious' in history and len(history['class_ious']) > 0:
    last_class_ious = history['class_ious'][-1]
elif 'avg_class_ious' in locals():
    last_class_ious = avg_class_ious # Use the variable from the last loop run

if 'last_class_ious' in locals():
    axes[1, 1].bar(range(len(Config.CLASS_NAMES)), last_class_ious, color='skyblue', edgecolor='black')
    axes[1, 1].set_xticks(range(len(Config.CLASS_NAMES)))
    axes[1, 1].set_xticklabels(Config.CLASS_NAMES, rotation=45, ha='right')
    axes[1, 1].set_ylabel('IoU')
    axes[1, 1].set_title('Final Per-Class IoU')
    axes[1, 1].grid(True, axis='y')

plt.tight_layout()
plt.savefig(f"{Config.OUTPUT_DIR}/training_curves.png", dpi=150, bbox_inches='tight')
plt.show()
print(f"✓ Curves saved to: {Config.OUTPUT_DIR}/training_curves.png")

# 2. Save History & Summary
history_path = f"{Config.OUTPUT_DIR}/training_history.json"
# Convert numpy arrays to lists for JSON serialization
json_history = {k: [float(v) for v in vals] if isinstance(vals, list) else vals for k, vals in history.items()}

with open(history_path, 'w') as f:
    json.dump(json_history, f, indent=2)
print(f"✓ History saved: training_history.json")

summary_path = f"{Config.OUTPUT_DIR}/training_summary.txt"
with open(summary_path, 'w') as f:
    f.write(f"Training Summary - {Config.RUN_NAME}\n")
    f.write("=" * 70 + "\n\n")
    f.write(f"Best Val IoU: {best_iou:.4f}\n")
    f.write(f"Total Epochs: {Config.NUM_EPOCHS}\n\n")
    f.write("Final Per-Class IoU:\n")
    if 'last_class_ious' in locals():
        for name, iou in zip(Config.CLASS_NAMES, last_class_ious):
            f.write(f"  {name:15s}: {iou:.4f}\n")
print(f"✓ Summary saved: training_summary.txt")

# 3. Create Final Backup ZIP (Directly to Drive)
print("\n" + "="*70)
print("CREATING FINAL BACKUP ZIP")
print("="*70)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# Save directly to the Drive folder we mounted earlier
final_zip_path = f"/content/drive/MyDrive/Duality_Project/FINAL_BACKUP_{Config.RUN_NAME}_{timestamp}.zip"

print(f"Zipping to Google Drive: {final_zip_path}...")

with zipfile.ZipFile(final_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add Best Model
    best_model_path = f"{Config.CHECKPOINT_DIR}/best_model.pth"
    if os.path.exists(best_model_path):
        zipf.write(best_model_path, arcname="best_model.pth")
        print(f"  ✓ Added: best_model.pth")

    # Add History & Summary
    zipf.write(history_path, arcname="training_history.json")
    zipf.write(summary_path, arcname="training_summary.txt")

    # Add Plots
    for plot_file in Path(Config.OUTPUT_DIR).glob("*.png"):
        zipf.write(plot_file, arcname=f"outputs/{plot_file.name}")
        print(f"  ✓ Added: {plot_file.name}")

final_size_mb = os.path.getsize(final_zip_path) / (1024 * 1024)

print("\n" + "="*70)
print("🎉 BACKUP COMPLETE!")
print("="*70)
print(f"📦 File: {os.path.basename(final_zip_path)}")
print(f"📊 Size: {final_size_mb:.2f} MB")
print(f"📁 Location: {final_zip_path}")
print(f"✓ You can find this file in your Google Drive folder 'Duality_Project'")